In [ ]:
import glob
from tqdm.auto import tqdm

X = []

for dataset in ["train", "test"]:
    for sentiment in ["neg", "pos"]:
        for file in tqdm(glob.glob(f"/kaggle/input/aclimdb/aclImdb/{dataset}/{sentiment}/*.txt")):
            with open(file, "r") as f:
                X.append({"text": f.read(), "dataset": dataset, "target": sentiment})

  0%|          | 0/12500 [00:00<?, ?it/s]

  0%|          | 0/12500 [00:00<?, ?it/s]

  0%|          | 0/12500 [00:00<?, ?it/s]

  0%|          | 0/12500 [00:00<?, ?it/s]

In [ ]:
!pip install optuna -q
!pip install catboost -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 11.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import optuna
import time
from collections import defaultdict
from tqdm.auto import tqdm
tqdm.pandas()

from sklearn.preprocessing import (OneHotEncoder, StandardScaler, MinMaxScaler,
                                   PolynomialFeatures, QuantileTransformer,
                                   PowerTransformer, LabelEncoder)
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import (roc_auc_score, classification_report, mean_absolute_error,
                             accuracy_score, f1_score, average_precision_score)
from sklearn.model_selection import (RepeatedStratifiedKFold, StratifiedKFold, KFold,
                                     train_test_split, cross_val_score)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.calibration import CalibratedClassifierCV
# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin

from catboost import CatBoostClassifier, CatBoostRegressor
# from category_encoders import CatBoostEncoder
from lightgbm import LGBMRegressor, LGBMClassifier
from xgboost import XGBRegressor, XGBClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from sklearn.ensemble import (RandomForestClassifier, RandomForestRegressor,
                              HistGradientBoostingRegressor,
                              HistGradientBoostingClassifier,
                              StackingClassifier, StackingRegressor,
                              VotingClassifier, VotingRegressor)
from sklearn.svm import LinearSVC, LinearSVR
from sklearn.linear_model import LogisticRegression, LinearRegression

In [ ]:
X = pd.read_csv("train_tokenized.csv")
X_test = pd.read_csv("test_tokenized.csv")

encoder = LabelEncoder()

y = encoder.fit_transform(X["target"])
X = X.drop(["target"], axis=1)

y_test = encoder.transform(X_test["target"])
X_test = X_test.drop(["target"], axis=1)

In [ ]:
np.random.seed(69)

def cb_objective(trial):
    params = {
#         'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'subsample': trial.suggest_float('subsample', 0.1, 1),
#         'colsample_bylevel': trial.suggest_float('colsample_bylevel', 5e-1, 1),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-2, 1e4),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 15),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'random_state': 69,
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", [
                "Poisson",
                "Bernoulli",
                "MVS"]
        ),
        'verbose': 0,
        "task_type": "GPU",
        'text_features': ['text']
    }
#     params = {
#         "iterations": 1000,
#         "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
#         "depth": trial.suggest_int("depth", 1, 10),
#         "subsample": trial.suggest_float("subsample", 0.05, 1.0),
#         "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
#         "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
#         'verbose': 0,
#         'text_features': ['text'],
#         }

#     'eval_set': (X_val, y_val),
#     "od_type": "Iter"

    model = CatBoostClassifier(**params)
    optuna_score = cross_val_score(model, X, y, scoring="accuracy", cv=5)

    return np.mean(optuna_score)

cb_study = optuna.create_study(direction = 'maximize')
cb_study.optimize(cb_objective, n_trials=100, show_progress_bar=True, n_jobs=1)

[I 2023-07-31 00:20:03,631] A new study created in memory with name: no-name-162310bc-1cda-4483-89a1-8b0625ae8acb


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2023-07-31 01:03:17,285] Trial 0 finished with value: 0.8738400000000001 and parameters: {'max_depth': 9, 'subsample': 0.7683331210668142, 'l2_leaf_reg': 6907.936079859354, 'min_child_samples': 12, 'n_estimators': 2692, 'bootstrap_type': 'Poisson'}. Best is trial 0 with value: 0.8738400000000001.
[I 2023-07-31 01:04:17,578] Trial 1 finished with value: 0.8462 and parameters: {'max_depth': 4, 'subsample': 0.5949678591727032, 'l2_leaf_reg': 9033.052297878889, 'min_child_samples': 13, 'n_estimators': 172, 'bootstrap_type': 'Bernoulli'}. Best is trial 0 with value: 0.8738400000000001.
[I 2023-07-31 01:05:18,727] Trial 2 finished with value: 0.53728 and parameters: {'max_depth': 8, 'subsample': 0.1905529413763274, 'l2_leaf_reg': 9485.01268213737, 'min_child_samples': 4, 'n_estimators': 187, 'bootstrap_type': 'MVS'}. Best is trial 0 with value: 0.8738400000000001.
[I 2023-07-31 01:43:07,128] Trial 3 finished with value: 0.90328 and parameters: {'max_depth': 9, 'subsample': 0.7632869033205

In [ ]:
cb_study.best_value

In [ ]:
cb_study.best_params